_Camille Avestruz (UChicago)_, _Alex Malz (NYU)_, _Tom McClintock (SBU)_ & Others

This is a wishlist for how we'd like to use `clmassmod`.  We will refactor the code to make this notebook run!

Our understanding of how folks currently use this code is based in part on the [Analytic Bias Estimate (ABE)](https://github.com/LSSTDESC/clmassmod/blob/master/notebooks/Analytic_bias_estimate.ipynb) and [Statistical Power Check (SPC)](https://github.com/LSSTDESC/clmassmod/blob/master/notebooks/Statistical%20Power%20Check.ipynb) demos.  There was a lot of other stuff here but we think it was more about simulating mock data, not the analysis pipeline we want to work on.  It can be found in commit `a103463`. 

In [ ]:
import clmassmod as clmm

`model_info` would be a dict with the name of the model and its parameters to support comparison of models, read in from a config file by the user/script outside the CLMassMod object.

In [ ]:
massmod = clm.CLMassMod(model_info)

A `CLMassMod` object should be able to perform fitting based on any combination of a variety of lensing observables, not just a mass map.

In [ ]:
massmod.obs_mass_map = in_data_mass_map
massmod.obs_kappa_map = in_data_kappa_map
massmod.obs_gamma_theta = in_data_gamma_theta
massmod.obs_delta_sigma = in_data_delta_sigma

The most basic functionality of `clmassmod` should be to fit cluster masses, saving data incrimentally as insurance against computational instability.

`calc_method` is a string keyword for curvefit vs. pymc vs. whatever, could be parallelized

`calc_params` is a dict of parameters necessary for the specified fitting method

`save_loc` should be the path to which output is saved, one file per halo

`fit_info` is the object produced by the fitter, like the pymc MC object, with properties like diagnostics already in it

`out_data_profiles` is pdfs or mles over cluster mass and metadata, preferably as a set of (named) tuples

In [ ]:
out_data_profiles, fit_info = massmod.fit_mass_profiles(calc_method, calc_params, save_loc)

Progress should be saved incrimentally, but the user should also be able to save it manually.

In [ ]:
massmod.save_mass_profiles(user_func)

If binning is desirable, the user should provide a scheme for assigning clusters to bins, which would be a function outside of `clmassmod`.

In [ ]:
bin_defs = np.linspace(bin_min, bin_max, n_bins)

def bin_one(out_data_profile):
# Consider a user-defined function that takes a mass profile and assigns a bin 
# based on a list or array containing bin endpoints.
#     ...
    return(bin_id)

def bin_all(mass_profiles):
# Consider a user-defined function that takes all the mass profiles provides a list of lists (one per bin) 
# of where they can be found based on their save locations
#     ...
    bin_member_locs = [] * len(bin_defs)
    for one__profile in mass_profiles:
        bin_id = bin_one(bin_defs, fluster_profile)
        halo_loc = one__profile.loc
        bin_members[bin_id].append(halo_loc)
#     ...
    return(bin_member_locs)

bin_member_locs = bin_all(out_data_profiles)

A `clmassmod` object could then be instantiated for each bin

In [ ]:
bin_massmod = clm.CLMassMod(model_info)

Read in the individual profile fits, so this can be done totally independently

In [ ]:
bin_mass_profile_data = bin_massmod.read(bin_loc)

Again, `calc_params` is a dict including the name of the fitting method and any tunable kw parameters it needs.

In [ ]:
clust_mass_dist = bin_massmod.ensemble_fit(calc_params)

Now we can find the probability distribution over mass bias!

`truth` is the masses from simulations in some as yet unspecified format

In [ ]:
likelihoods, posteriors, mass_dist_bias_samps, mass_dist_scatter_samps = bin_massmod.compare(truth)